# SEAEXPLORER DATA PROCESSING CHAIN

2021/09/07 - Bastien Queste

# PACKAGES : 

In [ ]:
import importlib, os, gsw, gc

import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from scipy.optimize import fmin
from tqdm import tqdm

import SXBQ as sx

import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
%matplotlib widget

importlib.reload(sx)

# SETTINGS :

In [ ]:
missions = {'SEA063':np.arange(17,23)}
pqt_files = []
pqt_folders = []
for k in missions.keys():
    for m in missions[k]:
        pqt_files.append('D:/Storage/Dropbox/Jupyter/Data/Bornholm_'+k+'_M'+str(m)+'.pqt')
        pqt_folders.append('D:/Storage/Dropbox/VOTO_Data/7_SAMBA_004/SEA063_PLD089/SEA063_M'+str(m)+'\2_Sorted\4_PLD_raw')
        pqt_folders.append('D:/Storage/Dropbox/VOTO_Data/7_SAMBA_004/SEA063_PLD089/SEA063_M'+str(m)+'\2_Sorted\2_NAV')
pqt_files

In [ ]:
todo = {
    'convert_to_pqt': False, #'D:/Storage/Dropbox/Jupyter/', # False or list of directories to process
    'merge_pqt': False, # False or list of .pqt to load
    'RBR_corr': True, # True = regress, False = use raw, 1x4 array of coefficients = use supplied coefficients
    'flight_model': True,
    'process_adcp':False,
    'save_file': 'D:/Storage/Dropbox/Jupyter/Data/Bornholm_SEA055_M24.pqt' #'D:/Storage/Dropbox/Jupyter/Data/Bornholm_055_061_combined.pqt' # False or file path
}

# CONVERT TO PARQUET FROM RAW : 

In [ ]:
importlib.reload(sx)

if todo['convert_to_pqt']:
    print('Converting data to parquet...')
    def convert_to_pqt():
        data = sx.sxdf(todo['convert_to_pqt'])
        data.save(todo['save_file'])
    convert_to_pqt()
    gc.collect()
else:
    print('Skipping conversion to parquet.')

# MERGE PQT FILES : 

In [ ]:
if todo['merge_pqt']:
    print('Loading parquet files...')
    def merge_pqt():
        data = sx.sxdf()
        for k,save_file in enumerate(todo['merge_pqt']):
            _tmp = pd.read_parquet(save_file)
            _tmp['diveNum'] = _tmp['diveNum'] + (k * 1e6)
            data.data = data.data.append(_tmp,ignore_index=True)
            gc.collect()

        data.save(todo['save_file'])
    
    merge_pqt()
    gc.collect()
else:
    print('Skipping parquet file loading.')

# LOAD
Runs in all cases as previous ones save and clear.

In [ ]:
gc.collect()
data = sx.sxdf()
data.data = pd.read_parquet(todo['save_file'])

def delimitateProfiles():

    #data.data.sort_values('Timestamp', ignore_index=True, inplace=True)
    data.median_resample()

    _gd = np.isfinite(data.data['diveNum'].values)
    _, _tmp = np.unique( np.round(data.data['diveNum'].values[_gd]) ,return_inverse=True)

    data.data.loc[_gd,'diveNum'] = np.round(_tmp+1)
    data.data['diveNum'] = data.data['diveNum'].interpolate('nearest')
        
    data.data['profileNum'] = data.data['diveNum'].values*2
    _tmp = data.data['NAV_RESOURCE'].interpolate('nearest').values
    ind = (_tmp == 100) | (_tmp == 110) | (_tmp == 116)
    data.data.loc[ind,'profileNum'] = data.data.loc[ind,'profileNum'] - 1

delimitateProfiles()

gc.collect()

data.data

# PRESSURE CORRECTION : 

In [ ]:
data.data['pressure'] = data.data['LEGATO_PRESSURE'].interpolate('index')

# RBR CORRECTION :

In [ ]:
if todo['RBR_corr']:
    print('Correcting RBR data...')
    
    def RBR_correction():
        
        if 'speed' in data.data:
            flowSpeed = data.data.speed.interpolate('index').values
        else:
            print('Estimating flow speed...')
            flowSpeed = np.abs(np.gradient(data.data.pressure,sx.date2float(data.data.Timestamp)) / np.sin(np.deg2rad(data.data.Pitch.interpolate('index')))).fillna(value=0.0001)
            flowSpeed[flowSpeed > 1] = 1
            
        print('Aligning C-T...')
        time = sx.date2float(data.data['Timestamp'])
        temp = data.data['LEGATO_TEMPERATURE'].values
        _gd = np.isfinite(time+temp)
        lag = -0.92 * flowSpeed + 1.22 # from RBR
        _interp = interp1d(time[_gd] - lag[_gd], temp[_gd], bounds_error=False, fill_value=np.NaN)
        data.data['temperature'] = _interp(time)

        print('Performing thermal mass correction...')
        Fs = np.mean(1/np.gradient(time))
        print('         Assuming a sampling frequency of '+str(Fs)+' Hz.')
        

        def _calcSal(temp, cond, pres, coefs):
            a_offset = coefs[0]
            a_slope = coefs[1]
            t_offset = coefs[2]
            t_slope = coefs[3]
            alpha = a_offset + a_slope / flowSpeed
            tau = t_offset + t_slope / np.sqrt(flowSpeed)
            #tau = 11 # Parameter for Lueck and Picklo (1990)
            #alpha = 0.57/tau + 0.03122 # Parameter for Lueck and Picklo (1990)
            
            alpha[~np.isfinite(alpha)] = a_offset
            tau[~np.isfinite(tau)] = t_offset
            
            beta = 1/tau # Parameter for Lueck and Picklo (1990)
            fn = Fs/2 # Nyquist frequency

            a = 4*fn*alpha*tau/(1+4*fn*tau) # Parameter for Lueck and Picklo (1990)
            b = 1-2*a/alpha # Parameter for Lueck and Picklo (1990)

            # Compute temperature correction to obtained water temperature within the conductivity cell (Morison, 1994)
            _internal_bias = np.full_like(temp,0)

            for sample in np.arange(1,len(_internal_bias)):
                # Recursive filter from Morison (1994)
                # if np.isfinite(temp[sample-1]):
                _internal_bias[sample] = -b[sample] * _internal_bias[sample-1] + a[sample] * (temp[sample] - temp[sample-1])
            return gsw.SP_from_C(cond, temp + _internal_bias, pres) # Practical salinity
                
        def _regressSal():
            _dives = np.unique(data.data.diveNum)[np.linspace(0, len(np.unique(data.data.diveNum))-1, 20).astype('int')] # Number of dives to regress over, 10 min, 100 probably good.
            _dives = (np.isin(data.data.diveNum, _dives)) & (np.isfinite(data.data.temperature))
            _temp = data.data.temperature[_dives]
            _cond = data.data.LEGATO_CONDUCTIVITY[_dives]
            _pres = data.data.pressure[_dives]
            
            # Use these generic values to scale parameters to same order of magnitude to help fmin.
            scaler = np.array([0.0135, 0.0264, 7.1499, 2.7858])
            
            def _PolyArea(x,y):
                _gg = np.isfinite(x+y)
                return 0.5*np.abs(np.dot(x[_gg],np.roll(y[_gg],1))-np.dot(y[_gg],np.roll(x[_gg],1)))
        
            def _scoreFunction(x_vals):
                return _PolyArea( _calcSal(_temp, _cond, _pres, x_vals*scaler) , _temp)
            
            print('Beginning regression (slow)...')
            print('         Initial minimisation score: '+str(_scoreFunction([1,1,1,1]))+'.')
            R = fmin(_scoreFunction, [1,1,1,1], disp=False, full_output=True, maxiter=200)
            print('         Final minimisation score: '+str(_scoreFunction(R[0]))+'.')
            return R[0]*scaler
        
        if todo['RBR_corr'] is True:
            coefs = _regressSal()
            print('Regressed coefficients:')

        else:
            coefs = todo['RBR_corr']
            print('Correcting using supplied coefficients:')
        
        print(coefs)
        print('Applying correction to all data (slow)...')
        # We have to interpolate temperature here as it's a recursive filter and NaNs propagate.
        # Both temperature and the alpha and tau parameters have to be finite.
        # This means interpolate and fill temperature and flowSpeed, 
        # and avoid divisions by zero (with flowSpeed = 0 or pitch = 0)
        data.data['salinity'] = _calcSal(
            data.data.temperature.interpolate('index').fillna(method='backfill'), 
            data.data.LEGATO_CONDUCTIVITY, 
            data.data.pressure, 
            coefs)
        
        data.data.loc[~(abs(data.data['salinity']-data.data['LEGATO_SALINITY']) < 0.5),'salinity'] = np.NaN  # Can definitely play around with this threshold
        
    RBR_correction()
    
    print('Done.')
else:
    print('Using raw temperature and salinity')
    data.data.rename(columns={
        "LEGATO_TEMPERATURE": "temperature", 
        "LEGATO_SALINITY": "salinity"
        },inplace=True)

gc.collect()

In [ ]:
if todo['RBR_corr']:
    def verifyTScorr():
        %matplotlib widget
        plt.rcParams["figure.figsize"] = (7,7)
        fig = plt.figure()
        ax = fig.add_subplot(1, 1, 1)
        line, = ax.plot([0], [0], 'b-',linewidth=0.5)
        line2, = ax.plot([0], [0], 'r-',linewidth=0.5)
        plt.subplots_adjust(left=0.25, bottom=0.25)

        t = data.data.LEGATO_TEMPERATURE.interpolate('index')
        s = data.data.LEGATO_SALINITY.interpolate('index')
        tc = data.data.temperature.interpolate('index')
        sc = data.data.salinity.interpolate('index')
        dn = np.round(data.data.diveNum.interpolate('nearest'))

        XL = np.nanpercentile(s,[1,99])
        YL = np.nanpercentile(t,[1,99])
        plt.xlim(XL)
        plt.ylim(YL)

        plt.title('Red = corrected')

        def update(dnum = 1):
            line.set_xdata(s[dn == sdnum.val])
            line.set_ydata(t[dn == sdnum.val])

            line2.set_xdata(sc[data.data.diveNum == sdnum.val])
            line2.set_ydata(tc[data.data.diveNum == sdnum.val])

            fig.canvas.draw_idle()

        axdnum = plt.axes([0.25, 0.1, 0.65, 0.03])
        sdnum = Slider(axdnum, 'Dive num', 0, np.nanmax(data.data.diveNum), valinit=0, valstep=1)
        sdnum.on_changed(update)

        plt.show()

    verifyTScorr()

In [ ]:
if todo['RBR_corr']:
    def verifyTScorr():
        _u = np.remainder(data.data.profileNum,2) == 0
        _d = np.remainder(data.data.profileNum,2) == 1 ## VERIFY

        fig = plt.figure()

        plt.subplot(211)
        plt.scatter(data.data.salinity[_d],data.data.temperature[_d], 1,'r', alpha=0.01)
        plt.scatter(data.data.salinity[_u],data.data.temperature[_u], 1,'b', alpha=0.01)
        XL = np.nanpercentile(data.data.salinity,[0.1,99.9])
        YL = np.nanpercentile(data.data.temperature,[0.1,99.9])
        plt.xlim(XL)
        plt.ylim(YL)
        plt.title('Up vs Down - Corrected')

        plt.subplot(212)
        plt.scatter(data.data.LEGATO_SALINITY[_d],data.data.LEGATO_TEMPERATURE[_d], 1,'r', alpha=0.01)
        plt.scatter(data.data.LEGATO_SALINITY[_u],data.data.LEGATO_TEMPERATURE[_u], 1,'b', alpha=0.01)
        XL = np.nanpercentile(data.data.LEGATO_SALINITY,[0.1,99.9])
        YL = np.nanpercentile(data.data.LEGATO_TEMPERATURE,[0.1,99.9])
        plt.xlim(XL)
        plt.ylim(YL)
        plt.title('Up vs Down - Uncorrected')

    verifyTScorr()

# GSW BASICS :

In [ ]:
#Location
data.data['lon'] = sx.parseGPS(data.data.NAV_LONGITUDE).interpolate('index').fillna(method='backfill')
data.data['lat'] = sx.parseGPS(data.data.NAV_LATITUDE).interpolate('index').fillna(method='backfill')

# Basic:
data.data['sa'] = gsw.SA_from_SP(data.data['salinity'],data.data['pressure'],data.data['lon'],data.data['lat'])
data.data['ct'] = gsw.CT_from_t(data.data['sa'],data.data['temperature'],data.data['pressure'])

# Physical:
data.data['sigma0'] = gsw.sigma0(data.data['sa'], data.data['ct'])

# FLIGHT MODEL :

In [ ]:
importlib.reload(sx)
if todo['flight_model']:
    print('Running flight model regression...')
    
    def flight_model():      
        
        flight = sx.SlocumModel(
               data.data.Timestamp, 
               data.data.salinity.interpolate('index').values, 
               data.data.temperature.interpolate('index').values, 
               data.data.pressure.interpolate('index').values, 
               data.data.lon.interpolate('index').values, 
               data.data.lat.interpolate('index').values, 
               data.data.BallastPos.interpolate('index').values, 
               data.data.Pitch.interpolate('index').values, 
               data.data.profileNum.interpolate('nearest').values,
               data.data.NAV_RESOURCE.interpolate('nearest').values,
               mass=60.772) # Verify mass number !!

        flight.regress()
        
        data.data['alpha'] = flight.alpha
        data.data['speed'] = flight.speed
        data.data['speed_vert'] = flight.speed_vert
        data.data['speed_horz'] = flight.speed_horz
        data.data['w_H2O'] = flight.w_H2O
        
        return flight
       
    flight = flight_model()
    ind = flight._valid

In [ ]:
if todo['flight_model']:
    def verifyFlightModel():
        ## VISUALISATIONS:
        plt.rcParams["figure.figsize"] = (12,10)
        plt.figure() 

        plt.subplot(321)
        V,XI,YI = sx.grid2d(data.data.loc[ind,'profileNum'],data.data.loc[ind,'pressure'],data.data.loc[ind,'w_H2O'], xi=1, yi=1)
        plt.pcolor(XI/2,YI,V,shading='auto',cmap='coolwarm')
        plt.colorbar()
        plt.clim([-0.02,0.02])
        #plt.ylim([0,300])
        plt.gca().invert_yaxis()
        plt.title('W_H2O')
        
        plt.subplot(323)
        plt.plot(XI[:,1],np.nansum(V,axis=1))

        plt.subplot(322)
        _ = plt.hist(data.data.loc[ind,'w_H2O'],np.linspace(-0.1,0.1,200))
        ylim = plt.ylim()
        for pt in np.nanpercentile(data.data.loc[ind,'w_H2O'],[5,25,50,75,95]):
            plt.plot([pt,pt],ylim,':k')
        plt.plot([0,0],ylim,'-r')
        plt.title('W_H2O')
        plt.xlim([-0.05,0.05])

        ind2 = np.remainder(data.data['profileNum'],2) == 1 & ind
        D,XI,YI = sx.grid2d(data.data.loc[ind2,'diveNum'],data.data.loc[ind2,'pressure'],data.data.loc[ind2,'w_H2O'], xi=1, yi=3)
        ind2 = np.remainder(data.data['profileNum'],2) == 0 & ind
        U,XI,YI = sx.grid2d(data.data.loc[ind2,'diveNum'],data.data.loc[ind2,'pressure'],data.data.loc[ind2,'w_H2O'], xi=1, yi=3)

        plt.subplot(324)
        plt.pcolor(XI,YI,D-U,shading='auto',cmap='coolwarm')
        plt.colorbar()
        plt.clim([-0.02,0.02])
        #plt.ylim([0,300])
        #plt.xlim([0,60*60*24*2])
        plt.gca().invert_yaxis()
        plt.title('Down - Up W_H2O')
        
        plt.subplot(325)
        plt.pcolor(XI,YI,D,shading='auto',cmap='coolwarm')
        plt.colorbar()
        plt.clim([-0.02,0.02])
        #plt.ylim([0,300])
        #plt.xlim([0,60*60*24*2])
        plt.gca().invert_yaxis()
        plt.title('Downcasts')
        
        plt.subplot(326)
        plt.pcolor(XI,YI,U,shading='auto',cmap='coolwarm')
        plt.colorbar()
        plt.clim([-0.02,0.02])
        #plt.ylim([0,300])
        #plt.xlim([0,60*60*24*2])
        plt.gca().invert_yaxis()
        plt.title('Upcasts')
        
    verifyFlightModel()

In [ ]:
data.save(todo['save_file'])

In [ ]:
plt.close('all')
gc.collect()

# NEXT STEPS .... ? :